In [11]:
import pandas as pd
import requests
from trade import pymt5, settings

In [2]:
def list_options(symbol, expiration):
    """List options from https://opcoes.net.br/ API by expiration date"""
    url = f"https://opcoes.net.br/listaopcoes/completa?idAcao={symbol}&listarVencimentos=false&cotacoes=true&vencimentos={expiration}"
    r = requests.get(url).json()
    l = [ 
        [symbol, expiration, i[0].split("_")[0], i[2], i[3], i[5]] 
        for i in r["data"]["cotacoesOpcoes"]
        ]
    return pd.DataFrame(
        l, columns=["symbol", "expiration", "option", "type", "model", "strike"]
        )

In [3]:
def list_all_options(symbol):
    """List all options from https://opcoes.net.br/ API """
    url = f"https://opcoes.net.br/listaopcoes/completa?idAcao={symbol}&listarVencimentos=true&cotacoes=true"
    r = requests.get(url).json()
    expirations = [i["value"] for i in r["data"]["vencimentos"]]
    df = pd.concat([list_options(symbol, expiration) for expiration in expirations])
    return df

In [4]:
def options(symbol, spot_price, call_expire = "C", put_expire = "O"):
    """Returns a df with options that strike is aroud 10 % spot price"""
    option = symbol[0:4]
    delta = spot_price * 0.1
    df = list_all_options(symbol)
    df = df[(df['option'].str.contains(option + call_expire)) |
    (df['option'].str.contains(option + put_expire))]
    df = df[(df['strike'] > spot_price - delta) &
    (df['strike'] < spot_price + delta)]
    return df

In [20]:
pymt5.conecta_mt5("BTG")
last = pymt5.last_tick(settings.OPTIONS_WATHC)
df_options = pd.DataFrame()
for key in last.keys():
    df = options(key, last[key])
    df_options = pd.concat([df_options, df])
    

Informação: trade allowed - True


In [21]:
df_options

,symbol,expiration,option,type,model,strike
37,PETR4,2023-03-17,PETRC153,CALL,A,24.51
39,PETR4,2023-03-17,PETRC156,CALL,E,24.26
43,PETR4,2023-03-17,PETRC164,CALL,A,24.01
58,PETR4,2023-03-17,PETRC208,CALL,E,23.76
63,PETR4,2023-03-17,PETRC233,CALL,A,26.01
...,...,...,...,...,...,...
137,BOVA11,2023-03-17,BOVAO990,PUT,E,99.00
0,BOVA11,2024-03-15,BOVAC110,CALL,A,110.00
1,BOVA11,2024-03-15,BOVAC115,CALL,E,115.00
3,BOVA11,2024-03-15,BOVAO110,PUT,E,110.00
